# Knowledge Distillation

В этом ноутбуке содержится описание моих основных экспериментов по knowledge distillation. Здесь будут в основном графики и описания экспериментов, весь код для обучения и оценки качества моделей вынесен в отдельные модули. Проще всего запустить в Google Colab, тогда не нужно дополнительно устанавливать никаких зависимостей. Если хотите запустить решение на своей локальной машине, то следуйте инструкциям в README.md.

Выполните следующие две ячейки, если работаете в Google Colab:

In [ ]:
! git clone github.com/stdereka/knowledge-distillation
! cp -r knowledge-distillation/dark_knowledge/ .
! cp -r knowledge-distillation/experiments/ .

In [ ]:
import sys
sys.path.insert(1, "/content/knowledge-distillation")

## 1. Описание задачи

Метод KD был изначально предложен в [статье](https://arxiv.org/pdf/1503.02531.pdf). Для задачи многоклассовой классификации утверждается, что вероятности классов, предсказанные моделью-учителем, могут быть исползованы для обучения более простой модели-студента с меньшим числом параметров путём модификации лосса. Для многоклассовой классификации предлагается следующая функция потерь (здесь индекс $i$ пробегает по всем классам и по всем объектам):

\begin{equation}
E=-T^{2} (1 - \alpha) \sum_{i} \hat{y}_{i}(\mathbf{x} \mid T) \log y_{i}(\mathbf{x} \mid T)- \alpha\sum_{i} \bar{y}_{i} \log y_{i}(\mathbf{x} \mid 1)
\end{equation}

где $y_{i}(\mathbf{x} \mid T)=\frac{e^{\frac{z_{i}(\mathbf{x})}{T}}}{\sum_{j} e^{\frac{z_{j}(\mathrm{x})}{T}}}$ - предсказания модели при температуре $T$ (она требуется для того, чтобы предсказания сильнее отличались от 0 и 1 и несли больше информации), $\hat{y}_{i}(\mathbf{x} \mid T)$ - "мягкие" метки учителя, $\bar{y}_{i}$ - "жёсткие" groundtruth метки. $\alpha$ - весовой параметр для регулирования вклада жёсткой и мягкой кроссэнтропий в градиент. Авторы статьи пишут, что всё должно хорошо работать при $\alpha$ близком к нулю.


## 2. Почему Imagewoof?

Imagewoof - датасет из изображений собак, относящимся к разным породам. На нём можно решать задачу многоклассовой классификации. Я выбрал его для экспериментов по нескольким причинам. Во-первых, классы трудноразделимы, слишком простые модели не смогут эффективно решить задачи, поэтому есть шансы пронаблюдать эффекты KD на более сложных моделях. Во-вторых, по Imagewoof не так много публикаций на тему KD, это значит, что результаты экспериментов мне заранее не известны.

Сначала я хотел начать с воспроизведения результатов статьи на датасете MNIST, потом попробовать CIFAR-10, но это уже сделали за меня, например, [тут](https://github.com/peterliht/knowledge-distillation-pytorch). Я решил сразу начать со своих экспериментов на более сложной задаче.

## 3. Общая схема эксперимента

1. **Выбрать и обучить модель-учитель.** Эта модель должна содержать большое число параметров (возможно, даже избыточное) и быть сильно регуляризована (в задачах CV подойдут сильные дропауты и аугментации), а также выдавать очень хорошее (по меркам задачи) качество на отложенной выборке.

2. **Обучение и оценка качества модели-студента с Distillation Loss.** Модель-студент содержит ощутимо меньше параметров, чем модель-учитель и никак не регуляризована. На этом этапе можно играть с параметрами $\alpha$, $T$, размером обучающей выборки и много чем ещё.

3. **Обучить модель-студента без помощи учителя.** Здесь очень важно, чтобы процесс обучения отличался только лоссом, это позволит исключить вклад всех прочих факторов.

4. **Сравнить метрики в пунктах 2 и 3**, построить выводы об эффективности KD в текущем эксперименте.

Из бесплатных ресурсов в моём распоряженни был только Google Colab, это очень неудобно с точки зрения разработки и воспроизводимости результатов, но я смог найти выход и прийти к следующей схеме разработки. Код пишется и тестируется локально на моём компьютере. Когда всё готово, ноутбук с параметрами отправляется на Google Сolab, подтягивает код с GitHub и выполняет эксперимент. Затем код, результаты и параметры сохраняются в одном коммите, это позволяет при необходимости вернуться и воспроизвести эксперимент.

## 4. Демонстрационный эксперимент

В этой части я проведу эксперимент согласно пункту 3.

In [ ]:
# Load data
! wget https://s3.amazonaws.com/fast-ai-imageclas/imagewoof2-320.tgz
! tar zxf imagewoof2-320.tgz
# Check GPU model
! nvidia-smi

In [16]:
import torch
import random
import numpy as np
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader
from torch import nn
import matplotlib.pyplot as plt
import os
from training import train, predict, DistillationLoss
from models import *
from datasets import Imagewoof
from itertools import product

In [17]:
"""
Define globals and seed whatever I can seed
"""
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('Training on CPU')
    DEVICE = torch.device("cpu")
else:
    print('Training on GPU')
    DEVICE = torch.device("cuda")

SEED = 0
seed_everything(SEED)

TRAIN_DIR = Path('./imagewoof2-320/train')
TEST_DIR = Path('./imagewoof2-320/val')

train_val_files = sorted(list(TRAIN_DIR.rglob('*.JPEG')))
test_files = sorted(list(TEST_DIR.rglob('*.JPEG')))

train_val_labels = [path.parent.name for path in train_val_files]
LABEL_ENCODER = LabelEncoder()
LABEL_ENCODER.fit(train_val_labels)

N_CLASSES = LABEL_ENCODER.classes_.shape[0]

Training on CPU


### 4.1. Выбор и обучение модели-учителя

Учитель - Resnet101 с предобученными весами. Я добавил два скрытых полносвязных слоя по 2048 юнитов. Такая модель явно избыточна, очень быстро переобучается и требует адский дропаут ($p=0.95$), чтобы не переобучиться и показать хорошие результаты на валидационной выборке. При обучении этой модели я также задействовал сильные аугментации.

Для дальнейших экспериментов (например, с размером обучающей выборки для модели-студента) могут понадобиться предсказания учителя на всей обучающей выборке. Чтобы не допустить утечки данных я использую 1x4 кроссвалидацию со стратификацией по классам, итоговые предсказания - это out-of-fold предсказания моделей-учителей, соответствующих разным фолдам. Таким образом, для получения предсказаний на всём датасете нужно обучить учителя целых 4 раза, зато после этого можно надолго о нём забыть и заняться непосредственно KD.

In [ ]:
N_SPLITS = 4

seed_everything(SEED)

cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
ground = np.array(Imagewoof(train_val_files, LABEL_ENCODER).labels)

# Array for storing OOF logits
oof_logits = np.empty((len(train_val_files), N_CLASSES), float)

labels = LABEL_ENCODER.transform(train_val_labels)

test_dataset = Imagewoof(test_files, LABEL_ENCODER)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=64)

# Array for storing test dataset logits
test_logits = np.zeros((len(test_dataset), N_CLASSES), float)

for fold, (train_idx, val_idx) in enumerate(cv.split(train_val_files, labels)):
    print('Training on fold', fold + 1)

    val_dataset = Imagewoof(np.array(train_val_files)[val_idx], LABEL_ENCODER)
    train_dataset = Imagewoof(np.array(train_val_files)[train_idx], LABEL_ENCODER, augs=True)
    
    model = resnet101_teacher(N_CLASSES, DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=0.0003)
    criterion = nn.CrossEntropyLoss()
    history = train(train_dataset, val_dataset, model=model, epochs=20,
                              batch_size=64, device=DEVICE, opt=opt, criterion=criterion)
    
    val_loader = DataLoader(val_dataset, shuffle=False, batch_size=64)
    labels_val = ground[val_idx]
    
    logits_val = predict(model, val_loader, DEVICE, logit=True)
    oof_logits[val_idx] = logits_val
    preds_val = np.argmax(logits_val, axis=1)
    
    # Prediction on test set is an average of N_SPLITS models
    test_logits += predict(model, test_loader, DEVICE, logit=True)/N_SPLITS
    
    print(f'Fold {fold + 1} accuracy score:', accuracy_score(labels_val, preds_val))

oof_preds = np.argmax(oof_logits, axis=1)
print('OOF accuracy score:', accuracy_score(ground, oof_preds))

# Save results
os.makedirs("./dark_knowledge", exist_ok=True)
np.save("./dark_knowledge/resnet101_train_imagewoof.npy", oof_logits)
np.save("./dark_knowledge/resnet101_test_imagewoof.npy", test_logits)

У датасета Imagewoof есть собственный [leaderboard](https://github.com/fastai/imagenette#imagewoof-leaderboard). Можно видеть, что наш учитель выдаёт неплохие результаты. Точность выше, чем у представленных моделей, так как у моей модели предобученные веса.

### 4.2. Модель-студент. Дистилляция знаний

Учитель - Resnet18 с предобученными весами и двумя дополнительными скрытыми полносвязными слоями по 148 юнитов. При обучении этой модели я не задействую ни дропаут, ни аугментации. Спойлер: это необходимо, потому что эффект от KD очень маленький, и чтобы его обнаружить, нужно исключить все прочие факторы, влияющие на качество модели.

In [ ]:
os.makedirs("./experiments/hyperparams", exist_ok=True)
seed_everything(SEED)

resnet18 = resnet18_student2(N_CLASSES, DEVICE)
trainable = get_number_of_params(resnet18, trainable=True)
total = get_number_of_params(resnet18, trainable=False)
print(f"{trainable} (of {total}) trainable params")
train_dataset = Imagewoof(train_val_files, LABEL_ENCODER, teacher_labels="./dark_knowledge/resnet101_train_imagewoof.npy")
test_dataset = Imagewoof(test_files, LABEL_ENCODER, teacher_labels="./dark_knowledge/resnet101_test_imagewoof.npy")
opt = torch.optim.Adam(resnet18.parameters(), lr=0.0003)
criterion = DistillationLoss(alpha=0.1, temperature=4.0)
history_resnet18 = train(train_dataset, test_dataset, resnet18, 50, 64, DEVICE, opt, criterion)
plot_training_history(history_resnet18)
np.save(f"./experiments/hyperparams/history_resnet18_T_4.0_alpha_0.1.npy", history_resnet18)

### 4.3. Обучение без помощи учителя и сравнение результатов

Повторим процедуру предыдущего пункта, только без использования меток модели-учителя.

In [ ]:
os.makedirs("./experiments/hyperparams", exist_ok=True)
seed_everything(SEED)

seed_everything(SEED)
resnet18 = resnet18_student2(N_CLASSES, DEVICE)
trainable = get_number_of_params(resnet18, trainable=True)
total = get_number_of_params(resnet18, trainable=False)
print(f"{trainable} (of {total}) trainable params")
train_dataset = Imagewoof(train_val_files, LABEL_ENCODER)
test_dataset = Imagewoof(test_files, LABEL_ENCODER)
opt = torch.optim.Adam(resnet18.parameters(), lr=0.0003)
criterion = nn.CrossEntropyLoss()
history_resnet18_no_teacher = train(train_data set, test_dataset, resnet18, 50, 64, DEVICE, opt, criterion)
plot_training_history(history_resnet18)
np.save(f"./experiments/hyperparams/history_resnet18_no_teacher.npy", history_resnet18_no_teacher)

## 5. Эксперимент с разными температурами

## 6. Результаты и промежуточные выводы